Привет! Меня зовут Мостовой Роман, я начинающий Data Scientist, и это мой отчет по проделанной работе над кейсом GPN-CUP 2023.

Хочу отдельно отметить стиль подачи задания, видно, что приложено много сил и фантазии. Спасибо за предоставленную возможность опробовать свои силы!

Итак, я начал с того, что загрузил 4 таблицы, из которых состоят данные:
- `transaction_df.parquet` — данные о транзакциях;
- `df_competitors.parquet` — данные о ценах конкурентов;
- `weather_df.parquet` — данные о погоде;
- `df_cost.parquet` — данные о себестоимости.

Оценил каждую из них и обнаружил следующее:
- начало среза: 2216-01-02;
- окончание среза: 2218-09-27;
- cтолбец `place` имеет 432 пропуска;
- столбец `amount` имеет нормальное распределение значений от -0.5 до 1.99, при этом отрицательные значения - это аномалии или ошибки;
- столбец `price` имеет много отрицательных выбросов;
- в столбце `cost` может быть недостаточно данных.

Было решено объединять все таблицы в одну с индексом по столбцу `date`. Для этого пришлось обработать все аномалии и пропуски. Пропуски в столбце `place` удалил, т.к. их замена заняла бы слишком много времени. Отрицательные значения в столбце `price` заменил на наиболее вероятные. При замене воспользовался правилом, что цена не может меняться в течении 3-х дней, а наиболее вероятная цена получается та, которая встречается больше всего раз в одной и той же комбинации даты, места и товара. Аномалию в столбце `amount` было решено исправить сдвигом всех значений на 0.5. *Применяя эти решения я старался сохранить максимальное количество данных. Все аномалии можно было удалить. Еще как вариант в столбце `amount`, все отрицательные значения можно было заменить на положительные. Если рассмотреть все возможные варианты замен, то у нас бы могло получится 6 разных датасетов.*  
Далее я сгруппировал ДатаФрейм `df_transactions` так, чтобы для каждой уникальной комбинации даты, продукта и места мы получили сумму значения столбца `amount`. *Тут я хотел добится чего-то вроде мультииндекса (дата, место, продукт), чтобы в дальнейшем было удобнее работь с данными.*  
Я выяснил, что у столбцов `hot`, `rain` и `snow` в ДатаФрейме `df_weather` всего 4 уникальных комбинации, а это значит, что каждый столбец - это булева переменная. Я переписал эти 3 столбца в 1 категориальный столбец `weather`, который будет принимать 4 возможных варианта ответа: `hot`, `rain`, `snow` и `clear`.  
В ДатаФрейме `df_competitors` выделил цены каждого конкурента в отдельный столбец и провели группировку по дате, товару и месту.  
Теперь все таблицы приведены к нужному виду и их можно объединять.  
Пропуски в столбце `cost` я заполнил при помощи интерполяции, а в столбцах с ценами конкурентов при помощи интерполяции и метода `bfill`. *Я пытался искать разные варианты как это сделать, но пропусков было настолько много, что я не смог придумать ничего лучше.*  
При графическом анализе себестоимости, обратили внимание на высокие амплетуды колебаний. *У меня есть подозрения на аномалии значений в столбце `cost`, но сейчас мне не хватает опыта, чтобы правильно их определить.*  
Создали дополнительные признаки `income` и `spread`:
- `spread` - наценка или маржа, это разница между `price` и `cost`; 
- `income` - чистая прибыль, это произведение `spread` и `amount`.  

Графически оценили чистую прибыль (`income`), отметили характерные для `cost` амплетуды колебания.  
Оценили стационарность с помощью теста ADF (расширенный тест Дики – Фуллера).  

Для обучения модели и прогнозирования цены, мы выбрали библиотеку Prophet. подготовили датасет, обучили модель на заданных параметрах, сделали предсказание и визуализировали результат. Определили точность модели с помощью среднеквадратичной ошибки (RMSE). Сравнили получившийся результат с предложенной Baseline моделью. На графике визуально определили, что цены, предсказанные с помощью библиотеки Prophet, выше постоянной цены из Baseline.  
С помощью функции, мы составили предсказание цены для каждого из 3-х продуктов в 5-ти городах на 90 дней вперед. И выгрузили это предсказание в файл `df_price_predicted.parquet`.

Увы, из-за болезни, мне не хватило времени, чтобы закончить эту работу...

Далее, я опишу то, на что мне не хватило времени и что я обызательно хочу доделать:

На данный момент предсказания `price`, я считаю, недостаточно для определения максимальной прибыли. Необходимо предсказать на 90 дней вперед цены каждого из конкурентов, возможно, создать еще один столбец со средней ценой конкурентов (не уверен, что как признак для обучения модели, это будет очень полезно, но стоит попробовать). А так же необходимо составить предсказание погоды (`weather`), себестоимости (`cost`) и объема продаж(`amount`).  
Еще у меня есть гипотеза, что если `cost` преобразовать с помощью скользящего среднего, то это не должно сильно отразиться на прибыли, зато должно улучшить точность предсказания модели.  
Итак, имея предсказания всех признаков на 90 дней вперед, можно лучше обучить модель и составить более точное предсказание по ценам. Я понимаю, что в каждом предсказании каждого признака будет содержаться какая-то ошибка и, чем больше у меня будет этих признаков, тем сильнее эта ошибка будет накапливаться. Этот аспект меня беспокоит.  
Необходимо проанализировать при каких условиях были получины максимальные чистые прибыли и постараться обучить этому модель. Альтернативная гипотеза у меня следующая: возможно, стоит предсказывать именно максимальную чистую прибыль, а уже потом имея предсказания объемов продаж и себестоимости, вычислять цены.  
Prophet - это хорошая универсальная библиотека для первичного анализа. Я же хочу оценить точность разных моделей: Decision Tree, Random Forest, LightGBM, CatBoost, нейросеть в несколько слоев. Но на графике цены хорошо читается линейность, поэтому, мне кажется, что линейные модели будут наиболее эффективны, например, Linear Regression или SVM.

Оценю точность всех моделей, выберу одну и постараюсь на ней составить более точный прогноз цены.